<a href="https://colab.research.google.com/github/wadeaT/Cloud-Computing-Course/blob/main/HW2/index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests beautifulsoup4

In [12]:
#My index
from bs4 import BeautifulSoup
from nltk.stem import PorterStemmer
from collections import defaultdict
import requests
import json
import nltk
import re
nltk.download('punkt')

class IndexCreator:
    #Class Initialization
    def __init__(self, start_url, firebase_url, max_pages=50):
        self.start_url = start_url # starting URL to proccess
        self.firebase_url = firebase_url.rstrip('/') + '/'  # the firebase URL
        self.max_pages = max_pages  # maximum number of pages to process
        self.index = defaultdict(dict) #for the index to automatically handle new terms
        self.processed_pages = 0 # number of proccessed pages, start at 0
        self.visited_urls = set() #visited URLS

        # Patterns for URLs we want to follow
        self.relevant_patterns = [
            # Core Compute Products
            r'/products/compute',
            r'/compute',  # General compute pages
            r'/kubernetes-engine',  # Google Kubernetes Engine (GKE)
            r'/container-optimized-os',
            r'/run',  # Cloud Run
            r'/functions',  # Cloud Functions
            r'/app-engine',  # App Engine

            # Compute Documentation
            r'/compute/docs',  # All compute docs
            r'/compute/docs/concepts',
            r'/compute/docs/instances',
            r'/compute/docs/disks',
            r'/compute/docs/networking',
            r'/compute/docs/security',
            r'/compute/docs/regions-zones',
            r'/compute/docs/quickstarts',
            r'/compute/docs/tutorials',
            r'/compute/docs/containers',

            # Container and Kubernetes related
            r'/kubernetes-engine/docs',
            r'/container-registry/docs',
            r'/containers',

            # Serverless Computing
            r'/serverless',
            r'/run/docs',  # Cloud Run documentation
            r'/functions/docs',  # Cloud Functions documentation
            r'/app-engine/docs',  # App Engine documentation

            # Infrastructure and VMs
            r'/vpc/docs',  # Virtual Private Cloud
            r'/virtual-machines',
            r'/vmware-engine',

            # Pricing and Planning
            r'/compute/pricing',
            r'/compute/docs/machine-types',
            r'/compute/docs/cpu-platforms',

            # Solutions and Reference
            r'/solutions/compute',
            r'/architecture/compute',
            r'/docs/cloud-computing'
        ]

    #checks if that pattern appears anywhere in the URL string
    #Returns True/False for each check
    def is_relevant_url(self, url):
        """Check if URL matches our relevant patterns"""
        return any(pattern in url for pattern in self.relevant_patterns)

    def extract_content(self, url):
        """Extract meaningful content from a page"""
        try:
            #makes an HTTP request to get the webpage content
            response = requests.get(url)
            #Uses BeautifulSoup to parse the HTML into a soup object for easy navigation
            soup = BeautifulSoup(response.text, 'html.parser')

            # Remove navigation, footer, etc.
            for elem in soup.find_all(['nav', 'footer', 'script', 'style']):
                elem.decompose()

            #extracting main content
            content_sections = []
            # Get main content
            main_content = soup.find('main')
            if main_content:
                content_sections.append(main_content.get_text())

            # Get article content
            article = soup.find('article')
            if article:
                content_sections.append(article.get_text())

            # Get section content
            sections = soup.find_all('section')
            for section in sections:
                content_sections.append(section.get_text())

            return ' '.join(content_sections)
        except Exception as e:
            print(f"Error extracting content from {url}: {e}")
            return ""

    def get_links(self, url):
        """Get relevant links from a page"""
        try:
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')
            links = set() #Using a set to avoid duplicate links

            #finds all <a> tags that have an href attribute
            for link in soup.find_all('a', href=True):
                href = link['href'] #Gets the href value (the URL)
                full_url = ''

                if href.startswith('http'):
                    if 'cloud.google.com' in href:
                        full_url = href
                elif href.startswith('/'):
                    full_url = f"https://cloud.google.com{href}"

                if full_url and self.is_relevant_url(full_url):
                    links.add(full_url)

            return list(links)
        except Exception as e:
            print(f"Error getting links from {url}: {e}")
            return []

    def process_text(self, text):
        """Process text into stemmed words"""
        stop_words = {'ha','thi','skip','-','&', 'me', 'my', 'myself', 'we', 'our', 'ours',
                     'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your',
                     'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself',
                     'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its'}

        words = []
        for word in text.split():
            word = word.lower()
            word = ''.join(c for c in word if c.isalnum())
            if word and word not in stop_words and len(word) > 2:
                words.append(word)
        #Uses Porter Stemming algorithm to reduce words to their root form
        stemmer = PorterStemmer()
        return [stemmer.stem(word) for word in words]

    def create_index(self):
        """Create inverted index from relevant pages"""
        queue = [self.start_url]
        doc_id = 1

        while queue and self.processed_pages < self.max_pages:
            url = queue.pop(0)

            if url in self.visited_urls:
                continue

            print(f"Processing page {self.processed_pages + 1}/{self.max_pages}: {url}")

            # Extract and process content
            content = self.extract_content(url)
            if content:
                words = self.process_text(content)

                # Count word occurrences
                word_counts = defaultdict(int)
                for word in words:
                    word_counts[word] += 1

                # Update index
                for word, count in word_counts.items():
                    self.index[word][str(doc_id)] = {
                        "url": url,
                        "counter": count
                    }

                # Mark as processed
                self.visited_urls.add(url)
                self.processed_pages += 1
                doc_id += 1

                # Get new links to process
                if self.processed_pages < self.max_pages:
                    new_links = self.get_links(url)
                    queue.extend([link for link in new_links if link not in self.visited_urls])

        print(f"\nIndexing complete!")
        print(f"Pages processed: {self.processed_pages}")
        print(f"Unique terms indexed: {len(self.index)}")

    def create_final_data(self):
        """Format index for database storage"""
        return [{
            'term': word,
            'DocId': docs
        } for word, docs in self.index.items()]

    def upload_to_firebase(self):
        """Upload index to Firebase using REST API"""
        print("\nUploading to Firebase...")

        data = self.create_final_data()
        index_url = f"{self.firebase_url}index.json"

        try:
            response = requests.put(index_url, json=data)
            response.raise_for_status()
            print("Upload complete! Data stored in Firebase.")

        except requests.exceptions.RequestException as e:
            print(f"Error uploading to Firebase: {e}")

    def search_word(self, term):
        """Search for a word in the index and return matching document IDs"""
        # Stem the search term to match the index
        stemmer = PorterStemmer()
        stemmed_term = stemmer.stem(term.lower())

        # Get matching documents from the index
        if stemmed_term in self.index:
            return list(self.index[stemmed_term].keys())
        return []

    def get_term_frequency(self, term, doc_id):
        """Get the frequency of a term in a specific document"""
        # Stem the term to match the index
        stemmer = PorterStemmer()
        stemmed_term = stemmer.stem(term.lower())

        # Return the term frequency if it exists, otherwise return 0
        if stemmed_term in self.index and doc_id in self.index[stemmed_term]:
            return self.index[stemmed_term][doc_id]["counter"]
        return 0

    def get_document_url(self, doc_id):
        """Get the URL for a specific document ID"""
        # Search through the index to find the document URL
        for term_data in self.index.values():
            if doc_id in term_data:
                return term_data[doc_id]["url"]
        return None



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
def create_and_upload_index(firebase_url):
    """Creates an index and uploads it to Firebase with the given Firebase URL"""
    # Static starting URL
    url = "https://cloud.google.com/products/compute"

    # Create the IndexCreator object
    indexer = IndexCreator(url, firebase_url, max_pages=50)

    # Create the index by processing the pages
    indexer.create_index()

    # Upload the generated index to Firebase
    indexer.upload_to_firebase()


In [4]:
from collections import defaultdict
import requests
from datetime import datetime
from nltk.stem import PorterStemmer
import nltk
nltk.download('punkt')

# First, get the index from Firebase
def get_index_from_db(firebase_url):
    # Ensure the URL is properly formatted
    firebase_url = firebase_url.rstrip('/')  # Remove trailing slash if present
    index_url = f"{firebase_url}/index.json"  # Add /index.json

    print(f"Attempting to fetch from: {index_url}")  # Debug print

    try:
        response = requests.get(index_url)
        response.raise_for_status()
        index_data = response.json()
        index = defaultdict(dict)

        for entry in index_data:
            term = entry['term']
            docs = entry['DocId']
            index[term] = docs

        print("Successfully retrieved index from database")
        return index
    except requests.exceptions.RequestException as e:
        print(f"Error retrieving index from Firebase: {e}")
        return None

# Initialize with the correct Firebase URL
firebase_url = "https://turtlenimbus-3cb66-default-rtdb.firebaseio.com"
create_and_upload_index(firebase_url)
index = get_index_from_db(firebase_url)

# Let's see what terms we have in the index
if index:
    print("\nSample of indexed terms:")
    sample_terms = list(index.keys())[:5]
    for term in sample_terms:
        print(f"Term: {term}")
        print(f"Number of documents: {len(index[term])}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Processing page 1/50: https://cloud.google.com/products/compute
Processing page 2/50: https://cloud.google.com/compute/docs/images/create-custom
Processing page 3/50: https://cloud.google.com/compute/docs/containers/deploying-containers
Processing page 4/50: https://cloud.google.com/functions/
Processing page 5/50: https://cloud.google.com/compute/sole-tenant-pricing
Processing page 6/50: https://cloud.google.com/compute/docs/create-linux-vm-instance
Processing page 7/50: https://console.cloud.google.com/compute/instances?_gl=1*qsgqme*_ga*MjA3OTc3Nzk3NS4xNzA0ODkxMjM2*_ga_4LYFWVHBEB*MTcwNTU1ODg4OC4yNy4xLjE3MDU1NjM3MjYuMC4wLjA.
Processing page 8/50: https://cloud.google.com/compute/docs/memory-optimized-machines#m3_machine_types
Processing page 9/50: https://cloud.google.com/compute/docs/machine-resource
Processing page 10/50: https://cloud.google.com/vmware-engine?hl=en
Processing page 11/50: https://cloud.google.com/compute/docs/general-purpose-machines#n2d_machines
Processing page 12/

In [ ]:
'''from firebase_admin import credentials, firestore
import firebase_admin

def initialize_firebase():
    """Initialize Firebase connection"""
    try:
        # Check if already initialized
        if not firebase_admin._apps:
            cred = credentials.Certificate({
                "type": "service_account",
                "project_id": "turtlenimbus-3cb66",
                # Add other credential details from your Firebase service account JSON
            })
            firebase_admin.initialize_app(cred)
        return firestore.client()
    except Exception as e:
        print(f"Error initializing Firebase: {e}")
        return None

# Initialize Firebase when notebook runs
db = initialize_firebase()'''


In [9]:
class QueryService:
    def __init__(self, index):
        self.index = index
        self.stemmer = PorterStemmer()
        self.queries = {}

    def search_word(self, term):
        """Search for a word in the index and return matching document IDs"""
        stemmed_term = self.stemmer.stem(term.lower())
        print(f"\nSearching for term: {term}")
        print(f"Stemmed form: {stemmed_term}")

        if stemmed_term in self.index:
            print(f"Term found in index")
            doc_data = self.index[stemmed_term]

            # Handle list structure
            if isinstance(doc_data, list):
                # Convert list to dictionary format
                doc_dict = {}
                for i, item in enumerate(doc_data):
                    if item is not None:  # Skip None entries
                        doc_dict[str(i)] = item
                return doc_dict
            # Handle dictionary structure
            elif isinstance(doc_data, dict):
                return doc_data

        print(f"Term not found in index")
        return {}

    def get_term_frequency(self, term, doc_id):
        """Get the frequency of a term in a specific document"""
        stemmed_term = self.stemmer.stem(term.lower())
        if stemmed_term in self.index:
            doc_data = self.index[stemmed_term]

            # Handle list structure
            if isinstance(doc_data, list):
                try:
                    doc_index = int(doc_id)
                    if 0 <= doc_index < len(doc_data) and doc_data[doc_index]:
                        return doc_data[doc_index].get('counter', 0)
                except (ValueError, IndexError):
                    pass
            # Handle dictionary structure
            elif isinstance(doc_data, dict) and doc_id in doc_data:
                return doc_data[doc_id].get('counter', 0)
        return 0

    def get_document_url(self, doc_id, term_data):
        """Get URL for a document from term data"""
        if isinstance(term_data, list):
            try:
                doc_index = int(doc_id)
                if 0 <= doc_index < len(term_data) and term_data[doc_index]:
                    return term_data[doc_index].get('url')
            except (ValueError, IndexError):
                pass
        elif isinstance(term_data, dict) and doc_id in term_data:
            return term_data[doc_id].get('url')
        return None

    def create_query(self, query_terms):
        """Create and execute a search query"""
        try:
            query_id = str(len(self.queries) + 1)
            print(f"\nProcessing query ID: {query_id}")
            print(f"Query terms: {query_terms}")

            # Find matching documents for all terms (AND logic)
            doc_sets = []
            term_data = {}  # Store term data for URL lookup

            for term in query_terms:
                doc_dict = self.search_word(term)
                if doc_dict:
                    doc_sets.append(set(doc_dict.keys()))
                    stemmed_term = self.stemmer.stem(term.lower())
                    term_data[stemmed_term] = self.index[stemmed_term]

            # If we have no results for any term, return empty
            if not doc_sets:
                print("No results found for any terms")
                return {"id": query_id, "message": "No results found for your query."}

            # Find intersection of all document sets
            results = set.intersection(*doc_sets)

            if not results:
                print("No documents match all terms")
                return {"id": query_id, "message": "No results found for your query."}

            # Rank results
            ranked_results = []
            print("\nRanking results...")

            for doc_id in results:
                # Calculate score as sum of term frequencies
                score = sum(self.get_term_frequency(term, doc_id) for term in query_terms)

                # Get URL from any term's data
                url = None
                for term_doc_data in term_data.values():
                    url = self.get_document_url(doc_id, term_doc_data)
                    if url:
                        break

                ranked_results.append({
                    "doc_id": doc_id,
                    "url": url,
                    "score": score
                })

            # Sort by score
            ranked_results.sort(key=lambda x: x["score"], reverse=True)
            print(f"Found {len(ranked_results)} matching documents")

            # Store query
            self.queries[query_id] = {
                'terms': query_terms,
                'results': ranked_results,
                'timestamp': str(datetime.now())
            }

            return {
                'id': query_id,
                'terms': query_terms,
                'results': ranked_results,
                'timestamp': str(datetime.now())
            }

        except Exception as e:
            print(f"Error during query processing: {str(e)}")
            return {'error': f"An error occurred while processing the query: {str(e)}"}

# Test code
def test_query_service(index):
    print("\n=== Testing Query Service ===")

    query_service = QueryService(index)
    test_query = ["cloud", "compute"]

    results = query_service.create_query(test_query)

    print("\nFinal Results:")
    if 'error' in results:
        print(f"Error: {results['error']}")
    elif 'message' in results:
        print(f"Message: {results['message']}")
    else:
        print(f"Query ID: {results['id']}")
        print(f"Found {len(results['results'])} results")
        print("\nTop 5 Results:")
        for result in results['results'][:5]:
            print(f"\nDocument ID: {result['doc_id']}")
            print(f"URL: {result['url']}")
            print(f"Score: {result['score']}")
            print("-" * 50)

In [11]:
import requests
from nltk.stem import PorterStemmer
from collections import defaultdict
from datetime import datetime
import nltk
nltk.download('punkt')

# Get index from Firebase
def get_index_from_db(firebase_url):
    firebase_url = firebase_url.rstrip('/')
    index_url = f"{firebase_url}/index.json"

    print(f"Fetching index from: {index_url}")

    try:
        response = requests.get(index_url)
        response.raise_for_status()
        index_data = response.json()
        index = defaultdict(dict)

        for entry in index_data:
            term = entry['term']
            docs = entry['DocId']
            index[term] = docs

        print(f"Successfully retrieved index with {len(index)} terms")
        return index
    except Exception as e:
        print(f"Error retrieving index: {e}")
        return None

class QueryService:
    def __init__(self, index):
        self.index = index
        self.stemmer = PorterStemmer()
        self.queries = {}

    def search_word(self, term):
        stemmed_term = self.stemmer.stem(term.lower())
        if stemmed_term in self.index:
            doc_data = self.index[stemmed_term]

            if isinstance(doc_data, list):
                doc_dict = {}
                for i, item in enumerate(doc_data):
                    if item is not None:
                        doc_dict[str(i)] = item
                return doc_dict
            elif isinstance(doc_data, dict):
                return doc_data

        return {}

    def get_term_frequency(self, term, doc_id):
        stemmed_term = self.stemmer.stem(term.lower())
        if stemmed_term in self.index:
            doc_data = self.index[stemmed_term]

            if isinstance(doc_data, list):
                try:
                    doc_index = int(doc_id)
                    if 0 <= doc_index < len(doc_data) and doc_data[doc_index]:
                        return doc_data[doc_index].get('counter', 0)
                except (ValueError, IndexError):
                    pass
            elif isinstance(doc_data, dict) and doc_id in doc_data:
                return doc_data[doc_id].get('counter', 0)
        return 0

    def get_document_url(self, doc_id, term_data):
        if isinstance(term_data, list):
            try:
                doc_index = int(doc_id)
                if 0 <= doc_index < len(term_data) and term_data[doc_index]:
                    return term_data[doc_index].get('url')
            except (ValueError, IndexError):
                pass
        elif isinstance(term_data, dict) and doc_id in term_data:
            return term_data[doc_id].get('url')
        return None

    def create_query(self, query_terms):
        try:
            query_id = str(len(self.queries) + 1)

            # Find matching documents
            doc_sets = []
            term_data = {}

            for term in query_terms:
                doc_dict = self.search_word(term)
                if doc_dict:
                    doc_sets.append(set(doc_dict.keys()))
                    stemmed_term = self.stemmer.stem(term.lower())
                    term_data[stemmed_term] = self.index[stemmed_term]

            if not doc_sets:
                return {"id": query_id, "message": "No results found for your query."}

            results = set.intersection(*doc_sets)

            if not results:
                return {"id": query_id, "message": "No results found for your query."}

            # Rank results
            ranked_results = []

            for doc_id in results:
                score = sum(self.get_term_frequency(term, doc_id) for term in query_terms)
                url = None
                for term_doc_data in term_data.values():
                    url = self.get_document_url(doc_id, term_doc_data)
                    if url:
                        break

                ranked_results.append({
                    "doc_id": doc_id,
                    "url": url,
                    "score": score
                })

            ranked_results.sort(key=lambda x: x["score"], reverse=True)

            return {
                'id': query_id,
                'terms': query_terms,
                'results': ranked_results,
                'timestamp': str(datetime.now())
            }

        except Exception as e:
            return {'error': f"An error occurred while processing the query: {str(e)}"}

# Simple test code
firebase_url = "https://turtlenimbus-d033a-default-rtdb.firebaseio.com/"
index = get_index_from_db(firebase_url)

if index:
    # Create query service
    query_service = QueryService(index)

    # Test with a simple query
    test_query = ["cloud", "compute"]
    print(f"\nTesting query: {test_query}")

    # Get results
    results = query_service.create_query(test_query)

    # Display results
    print("\nResults:")
    if 'error' in results:
        print(f"Error: {results['error']}")
    elif 'message' in results:
        print(f"Message: {results['message']}")
    else:
        print(f"Query ID: {results['id']}")
        print(f"Found {len(results['results'])} results")
        print("\nTop 5 Results:")
        for result in results['results'][:5]:
            print(f"\nDocument ID: {result['doc_id']}")
            print(f"URL: {result['url']}")
            print(f"Score: {result['score']}")
            print("-" * 50)
else:
    print("Failed to load index")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Fetching index from: https://turtlenimbus-3cb66-default-rtdb.firebaseio.com/index.json
Successfully retrieved index with 6056 terms

Testing query: ['cloud', 'compute']

Results:
Query ID: 1
Found 45 results

Top 5 Results:

Document ID: 46
URL: https://cloud.google.com/compute/docs/release-notes
Score: 1751
--------------------------------------------------

Document ID: 13
URL: https://cloud.google.com/compute/vm-instance-pricing#sustained_use
Score: 677
--------------------------------------------------

Document ID: 18
URL: https://cloud.google.com/compute/docs/images/create-custom
Score: 434
--------------------------------------------------

Document ID: 20
URL: https://cloud.google.com/compute/all-pricing
Score: 425
--------------------------------------------------

Document ID: 37
URL: https://cloud.google.com/compute/docs/instances/stop-start-instance#starting_a_stopped_instance
Score: 404
--------------------------------------------------
